In [ ]:
#from google.colab import drive
#drive.mount('/drive/content')

In [ ]:
!apt-get update -y

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 2s (128 kB/s)
Reading package lis

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null+

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
! ls

chess  sample_data		  spark-3.1.2-bin-hadoop2.7.tgz
drive  spark-3.1.2-bin-hadoop2.7  spark-3.1.2-bin-hadoop2.7.tgz.1


In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("sample").getOrCreate())

In [ ]:
import gdown

In [ ]:
gdown.download_folder(id='1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ', quiet=True, use_cookies=False)

['/content/chess/chess_schema.png',
 '/content/chess/chess_wc_history_game_info.csv',
 '/content/chess/chess_wc_history_moves.csv',
 '/content/chess/eco_codes.csv']

In [ ]:
history_data = spark.read.csv("/content/drive/MyDrive/chess/chess_wc_history_game_info.csv", inferSchema=True, header=True)
moves_data = spark.read.csv("/content/drive/MyDrive/chess/chess_wc_history_moves.csv", inferSchema=True, header=True)
eco_data = spark.read.csv("/content/drive/MyDrive/chess/eco_codes.csv", inferSchema=True, header=True)

In [ ]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [ ]:
history_data.groupBy('winner').count().show()

In [ ]:
history_data.groupBy('tournament_name').count().show()

In [ ]:
history_data.printSchema()

In [ ]:
history_data.show()

In [ ]:
moves_data.printSchema()

In [ ]:
moves_data.show()

In [ ]:
eco_data.printSchema()

In [ ]:
eco_data.show()

In [ ]:
print('size of the data:', history_data.count())
print('no of columns:', len(history_data.columns))
print('columns:', history_data.columns)

In [ ]:
# using pyspark for exploratory data analysis

import pyspark
from pyspark import sql
from pyspark.sql.functions import col

In [ ]:
history_data.groupBy('tournament_name').count().show(5000)

In [ ]:
moves_data.select('player').show()

In [ ]:
moves_data.groupBy('player').count().show()

# ***1. List of Winners of Each World champions Trophy.***

In [ ]:
from pyspark.sql.functions import col
winner_data = history_data.filter(col("winner") != "draw").show()

+--------------------+----------+------------------+------------+-----------+-----+------------------+------------------+------+---------+---------+-----------+-----------+------------------+----------+------------------+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|             event|        site|date_played|round|             white|             black|result|white_elo|black_elo|white_title|black_title|            winner|winner_elo|             loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+------------------+------------+-----------+-----+------------------+------------------+------+---------+---------+-----------+-----------+------------------+----------+------------------+---------+---------------------+---+--------------------+---------------+
|58e83255-93bb-4d5...|         6|          WCh 2021|   Dubai UAE| 2021.12.03|  6.0|         Carlsen,M|  Nepomniacht

In [ ]:
#history_data.filter("winner != 'draw'").select("winner").show()
#This table shows all the winners list of all competetion

In [ ]:
from pyspark.sql.functions import col

In [ ]:
#Here we are getting the actual results of winners of overall world championship
count = history_data.filter("winner != 'draw'").where(col('tournament_name').like("%W%")).select('winner', 'tournament_name')

In [ ]:
count.dropDuplicates(['winner', 'tournament_name']).show()

+--------------------+---------------+
|              winner|tournament_name|
+--------------------+---------------+
|   Steinitz, William| WorldChamp1890|
|           Kramnik,V| WorldChamp2006|
|             Anand,V| WorldChamp2010|
|  Botvinnik, Mikhail| WorldChamp1961|
|  Botvinnik, Mikhail| WorldChamp1948|
| Alekhine, Alexander| WorldChamp1929|
| Alekhine, Alexander| WorldChamp1937|
|           Kramnik,V| WorldChamp2007|
|              Leko,P| WorldChamp2004|
|         Tal, Mihail| WorldChamp1961|
|  Tarrasch, Siegbert| WorldChamp1908|
|           Gelfand,B| WorldChamp2012|
|             Anand,V| WorldChamp2012|
|      Kasparov, Gary| WorldChamp1984|
|  Botvinnik, Mikhail| WorldChamp1963|
|Janowsky, Dawid M...| WorldChamp1909|
|      Kasparov, Gary| WorldChamp1987|
|     Karpov, Anatoly| WorldChamp1978|
|  Botvinnik, Mikhail| WorldChamp1951|
|    Smyslov, Vassily| WorldChamp1948|
+--------------------+---------------+
only showing top 20 rows



# ***2.List of Players with number of times they have won Tournament in descending order(Max to min).***

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.functions import desc

In [ ]:
history_data.filter("winner != 'draw'").select("winner").groupBy("winner").count().sort(desc("count")).show()

+--------------------+-----+
|              winner|count|
+--------------------+-----+
|     Lasker, Emanuel|   52|
|     Karpov, Anatoly|   47|
|  Botvinnik, Mikhail|   46|
| Alekhine, Alexander|   43|
|   Steinitz, William|   43|
|           Topalov,V|   32|
|      Kasparov, Gary|   31|
|             Anand,V|   30|
|    Smyslov, Vassily|   24|
|            Adams,Mi|   20|
|      Kasimdzhanov,R|   19|
|          Grischuk,A|   17|
|      Adams, Michael|   16|
|           Svidler,P|   16|
|            Bareev,E|   15|
|            Shirov,A|   15|
|             Dreev,A|   15|
|   Chigorin, Mikhail|   14|
|Khalifman, Alexander|   14|
|           Euwe, Max|   14|
+--------------------+-----+
only showing top 20 rows



# ***3.Most and Least Popular eco move in world championship history.***

In [ ]:
history_data.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|winner|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|  1.0|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|  draw|      null| draw|     null|                    0|C88|2022-07-22T22:33:...| 

In [ ]:
moves_data.show(5)

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
eco_data.show(5)

+---+--------------------+------------------+--------+---------+
|eco|            eco_name|       eco_example|eco_type|eco_group|
+---+--------------------+------------------+--------+---------+
|A00|    Uncommon Opening|1 g4, a3, h3, etc.|       A|    Flank|
|A01|Nimzovich-Larsen ...|              1 b3|       A|    Flank|
|A02|      Bird's Opening|              1 f4|       A|    Flank|
|A03|      Bird's Opening|           1 f4 d5|       A|    Flank|
|A04|        Reti Opening|             1 Nf3|       A|    Flank|
+---+--------------------+------------------+--------+---------+
only showing top 5 rows



In [ ]:

combined_data = history_data.join(eco_data,history_data.eco ==  eco_data.eco,"inner") 


In [ ]:
combined_data.show(2)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+---+--------------------+--------------------+--------+-----------------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|winner|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|eco|            eco_name|         eco_example|eco_type|        eco_group|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+---+--------------------+--------------------+--------+-----------------+
|86e0b7f5-7b94-4ae.

In [ ]:
from pyspark.sql.functions import *

In [ ]:
import pyspark.sql.functions as F

In [ ]:
combined_data = combined_data.join(moves_data, combined_data.game_id == moves_data.game_id)

In [ ]:
combined_data.show(2)

+--------------------+----------+--------------------+---------+-----------+-----+----------------+--------------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+---+-----------------+--------------------+--------+--------------------+--------------------+-------+------------+--------------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----

In [ ]:
combined_data.columns